In [6]:
import numpy as np
from scipy.optimize import linprog
import scipy.sparse as spr
import itertools
from scipy.linalg import sqrtm
from jax.scipy.optimize import minimize
import jax.numpy as jnp
import jax
from itertools import product
import torch

In [7]:
# Set seed for reproducibility
torch.manual_seed(1)

# Define parameters
J = 500
K = 5
lambda_k = torch.ones(K)
phi_j = torch.normal(10, 300, size=(J, K))



# Define the function v using PyTorch
def v(B):
    if B.dtype == torch.bool:
        return phi_j[B, :].sum(0) @ lambda_k - B.sum()**2
    elif B.dtype == torch.int64:  # PyTorch uses int64 for integer tensors
        return phi_j[B, :].sum(0) @ lambda_k - len(B)**2
    else:
        print("Error: B must be a boolean or integer tensor")

In [8]:
def grad_v_hat(z_j):
    J = z_j.shape[0]  # Number of elements in z_j
    sorted_z_id = torch.argsort(z_j, descending=True)  # Sort indices in descending order
    
    grad = torch.zeros(J)  # Initialize the gradient array
    for j in range(J):
        # Compute the difference between v(sorted_z_id[:j+1]) and v(sorted_z_id[:j])
        grad[sorted_z_id[j]] = v(sorted_z_id[:j+1]) - v(sorted_z_id[:j])

    return grad

In [9]:
def mirror_descent(num_iterations, alpha):
    # Initialize z within the hypercube [0,1]^n
    torch.manual_seed(1)
    z = torch.ones(J) / 2  # Start with z initialized to 0.5 for all elements

    # Start Mirror Descent
    z_list = []
    for _ in range(num_iterations):
        grad = grad_v_hat(z)  # You need to ensure grad_v_hat is implemented using PyTorch

        # Update z using mirror descent
        z_new = z - alpha * grad / torch.norm(grad)  # Normalize the gradient
        z_new = torch.clamp(z_new, 0, 1)  # Clip values to stay within [0, 1]

        z = z_new
        z_list.append(z.clone())  # Store a copy of z

    # Calculate the final z_star by averaging over the last 1/30th of the iterations
    z_list_tensor = torch.stack(z_list)
    z_star = z_list_tensor[-int(torch.floor(torch.tensor(num_iterations) / 30)):].mean(0)
    
    # Round z_star and convert to boolean tensor
    bundle_star = z_star.round().bool()

    return z_star, bundle_star

In [14]:
# Example usage
num_iterations = 1000
alpha = 1
z_star, bundle_star = mirror_descent(num_iterations, alpha)

In [16]:
z_star[(z_star > 0.2) & (z_star < 0.8)]

tensor([0.6278])